In [1]:
import json
import openai
import datetime
from langame import LangameClient
c = LangameClient(path_to_config_file="../config.yaml")

In [2]:
from langame.conversation_starters import get_existing_conversation_starters
import logging
logger = logging.getLogger("generation")
memes, index, embeddings_model = get_existing_conversation_starters(
    c._firestore_client, logger=logger, confirmed=True)

Skipping id: 0ZcPZDcvuqgialj2wHpL, garbage
Skipping id: 0m2ltwf7wkxLzcHnFkQE, garbage
Skipping id: 1YWuwIp4srffwPNq7Aw9, garbage
Skipping id: 4XlXSjPhH6QSGZDzwIQe, garbage
Skipping id: 4sIjdMJr0exliT9yfvuL, garbage
Skipping id: 5MAMBAYySyu3lKrR3zLt, garbage
Skipping id: 6jNOw0e7VA6lHWddtqMy, garbage
Skipping id: 7yHXwGPlHpopQAHU3xf1, garbage
Skipping id: 8e9jVhvFpUIzj7ocNv06, garbage
Skipping id: 8tzYKdv2Hkwxsm8eNQgj, garbage
Skipping id: 9NdMRn8C285Eu3k4PdsS, garbage
Skipping id: 9ZfEEkrB3cjLooW6YZM2, garbage
Skipping id: B8hRGbGGOweqEyxuuplE, garbage
Skipping id: BH9ZtGe1g6q3KmK7oV2f, garbage
Skipping id: BZjXMPPKlmcqXG23i0mI, garbage
Skipping id: BwSIpHO7X8N1fhf5zTyr, garbage
Skipping id: C0yrKrOeX5kQKwEgyrJS, garbage
Skipping id: CYsOWtp1ed2XDeplB5O5, garbage
Skipping id: D8A4QcAlqidVArVPDKGP, garbage
Skipping id: DhuuWoHv7oGKSA5agAzZ, garbage
Skipping id: DjuEuISOCOG6R2D7ZlSy, garbage
Skipping id: DlXglThZUiPJcuyLLKHV, garbage
Skipping id: EBZCFk0aZPVS1SinJwcz, garbage
Skipping id

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'embeddings'

In [4]:
out_file_name = f"../data/fine_tune_generation_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

for e in memes:
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e['topics'])} ###",
            "completion": f" {e['content']}\n",
        }, outfile)
        outfile.write('\n')

In [5]:
!head -n3 $out_file_name | jq .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "prompt": "personal,relationship,relationships,social,big talk,personal growth ###",
  "completion": " Give eachother four praises and one critique\n"
}
{
  "prompt": "ice breaker ###",
  "completion": " When is a time when you know for sure you'll soon have to exchange a nice, meaningful conversation with someone new?\n"
}
{
  "prompt": "space exploration,space travel ###",
  "completion": " Do you think humans are the only intelligent life in the universe?\n"
}


In [6]:
!echo "execute: openai tools fine_tunes.prepare_data -f $(pwd)/$out_file_name"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
execute: openai tools fine_tunes.prepare_data -f /Users/louisbeaumont/Documents/langame-worker/notebooks/../data/fine_tune_generation_18_09_2022.jsonl


In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
train, test = train_test_split(
    open(f"{out_file_name.replace('.jsonl', '_prepared.jsonl')}").read().splitlines(), test_size=0.1)
with open(f"{out_file_name.replace('.jsonl', '')}_prepared_train.jsonl", "w") as outfile:
    outfile.write("\n".join(train))
with open(f"{out_file_name.replace('.jsonl', '')}_prepared_test.jsonl", "w") as outfile:
    outfile.write("\n".join(test))

In [8]:
!head -n3 {out_file_name.replace('.jsonl', '')}_prepared_train.jsonl | jq .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "prompt": "transhumanism,public welfare ###",
  "completion": " What do you think will happen if we continue to push the technological and social limits of humanity?\n"
}
{
  "prompt": "ice breaker ###",
  "completion": " What genre would your life story be if it was turned into a movie?\n"
}
{
  "prompt": "ice breaker,emotions ###",
  "completion": " If you could condense your own thoughts and feelings down to one word, what would that word be? Why?\n"
}


In [13]:
train_file = openai.File.create(
  file=open(f"{out_file_name.replace('.jsonl', '')}_prepared_train.jsonl", "rb"),
  purpose="fine-tune"
)
valid_file = openai.File.create(
  file=open(f"{out_file_name.replace('.jsonl', '')}_prepared_test.jsonl", "rb"),
  purpose="fine-tune"
)
ft = openai.FineTune.create(
    training_file=train_file["id"],
    validation_file=valid_file["id"],
    model="curie",
    suffix="generation",
)

In [19]:
from openai.wandb_logger import WandbLogger
import wandb
import re
values = open("../.env.production", "r").read()
key = re.findall(r"WANDB_KEY=\"(.*)\"", values)[0]
wandb.login(key=key, relogin=True)
WandbLogger.sync(
    id=ft["id"],
    project="generation",
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/louisbeaumont/.netrc


Fine-tune ft-x3ojk3Kxs2Gkz9SJhE4m1fK6 has the status "running" and will not be logged


'🎉 wandb sync completed successfully'

In [5]:
results = openai.FineTune.retrieve(
    ft["id"],
)
results

NameError: name 'ft' is not defined

In [8]:
!rm -rf _prepared*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
